# To do 
- add choosing gpus 
- add random seed for all ( from server code) ( add it to github) 
- implement baseline transformer  
- start train with it 
- connect with git cleaned version. The smartest version of my code and thoughts . 
- make report 

# Install libraries 


In [1]:
import sys, os


import sklearn
import mne
import wandb
sys.path.insert(1, os.path.realpath(os.path.pardir))



from nilearn import image
from nilearn import plotting

import numpy as np
import matplotlib.pyplot as plt

import mne
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d
from nilearn import datasets, image, masking
from nilearn.input_data import NiftiLabelsMasker


# animation part
from IPython.display import HTML
# from celluloid import Camera   # it is convinient method to animate
from matplotlib import animation, rc
from matplotlib.animation import FuncAnimation



## torch libraries 

import torch

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader, Subset

from pytorch_model_summary import summary


/home/user/.local/lib/python3.9/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
from utils import get_datasets
from utils import preproc
from utils import torch_dataset
from utils import train_utils
from utils import inference
from utils.models_arch import autoencoder_v3

# Set all hyperparameters
Parameters of dataset, random seed and so on 


# Upload dataset 
Upload EEG fMRI and labels 

In [5]:
patient = 'trio1'
path_to_dataset = '../data/eyes_open_closed_dataset/'
dataset_name = 'CWL' # 
remove_confounds = True
# common/koval_alvi/fmri_eeg_project/


df_eeg, df_fmri, labels_roi = get_datasets.download_cwl_dataset(patient, path_to_dataset, 
                                                                    remove_confounds=remove_confounds,
                                                                    verbose=True)
df_eeg_align, df_fmri_interp, fps = get_datasets.interpolate_df_eeg_fmri(df_eeg, df_fmri)

ALL path:  ../data/eyes_open_closed_dataset/trio1/CWL_Data/eeg/in-scan/trio1_mrcorrected_eoec_in-scan_hpump-off.set ../data/eyes_open_closed_dataset/trio1/CWL_Data/mri/epi_normalized/rwatrio1_eoec_in-scan_hpump-off.nii ../data/eyes_open_closed_dataset/trio1/CWL_Data/mri/epi_motionparams/rp_atrio1_eoec_in-scan_hpump-off.txt


/home/user/common/koval_alvi/fmri_eeg_project/utils/get_datasets.py:213: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = mne.io.read_raw_eeglab(eeg_path_set_file)
/home/user/common/koval_alvi/fmri_eeg_project/utils/get_datasets.py:213: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(eeg_path_set_file)
/home/user/common/koval_alvi/fmri_eeg_project/utils/get_datasets.py:171: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  motion_confound = pd.read_csv(motion_params_path, sep = '  ', header=None)


Dimension of our EEG data:  (303601, 31)
Dimension of our fMRi data:  (61, 72, 61, 146)
Dimension of our fMRi Roi data:  (143, 22)
fMRI info :  1.95
RoI:  ['Left Cerebral White Matter', 'Left Cerebral Cortex ', 'Left Lateral Ventricle', 'Left Thalamus', 'Left Caudate', 'Left Putamen', 'Left Pallidum', 'Brain-Stem', 'Left Hippocampus', 'Left Amygdala', 'Left Accumbens', 'Right Cerebral White Matter', 'Right Cerebral Cortex ', 'Right Lateral Ventricle', 'Right Thalamus', 'Right Caudate', 'Right Putamen', 'Right Pallidum', 'Right Hippocampus', 'Right Amygdala', 'Right Accumbens']


In [6]:
# patient = '32'
# path_to_dataset = '../data/NODDI_dataset_light/'
# dataset_name = 'NODDI'
# remove_confounds = True



# df_eeg, df_fmri, labels_roi = dataset_noddi.download_bids_noddi_dataset(patient, path_to_dataset, 
#                                                                     remove_confounds=remove_confounds,
#                                                                     verbose=True)
# df_eeg_align, df_fmri_interp, fps = dataset_noddi.interpolate_df_eeg_fmri(df_eeg, df_fmri)

# Process data
- divide on train and test 
- Process data separately
- apply bold delay correction

In [7]:
config = dict(  
                dataset_name = dataset_name,
                fps = fps,
                n_channels = 30,
                n_roi = 21,
                test_sec = 60, # in seconds.
                freqs = np.linspace(2, 100, 16), 
                
                bold_delay = 5,
                to_many = True,
                random_subsample = True,
                sample_per_epoch = 25600, 
                WINDOW_SIZE = 1024,
                    
                optimizer='adam',
                lr=0.001,
                weight_decay=0.00001, 
                batch_size=512, 

                loss_function = 'mse', 
                model_type = '1D_CNN_AE_changed'
                )


hp_autoencoder = dict(n_electrodes=config['n_channels'],
                      n_freqs = len(config['freqs']),
                      n_channels_out = config['n_roi'],
                      n_res_block=0, 
                      channels=[32, 32, 64, 64], 
                      kernel_sizes=[15, 11, 5, 5],
                      strides=[8, 8, 4]
                     )


config = {**hp_autoencoder, **config}

params_train = {'batch_size': config['batch_size'],
                'shuffle': True,
                'num_workers': 0}

params_val = {'batch_size': config['batch_size'],
              'shuffle': False}

In [8]:
df_eeg_align.shape

(276900, 31)

In [9]:
# delete time columns. 
# reshape time, ch -> [ch, time]


In [10]:
eeg_np = df_eeg_align.drop(['time'], axis=1).to_numpy().T
fmri_np = df_fmri_interp.drop(['time'], axis=1).to_numpy().T

dataset_train, dataset_test = preproc.divide_train_test(eeg_np, fmri_np,
                                                        fps=config['fps'], 
                                                        test_sec=config['test_sec'])



train_dataset_prep, freqs, inp_stats_train, out_stats_train = preproc.preproc_dataset(dataset_train, 
                                                                                      fps=config['fps'], 
                                                                                      freqs=config['freqs'], 
                                                                                      inp_means_stds=None, 
                                                                                      out_means_stds=None)
test_dataset_prep, freqs, inp_stats_test, out_stats_test = preproc.preproc_dataset(dataset_test,
                                                                                    fps = config['fps'],
                                                                                    freqs=config['freqs'],                                                                    
                                                                                    inp_means_stds=inp_stats_train, 
                                                                                    out_means_stds=out_stats_train)


# apply time dealy corrected
train_dataset_prep = preproc.bold_time_delay_align(train_dataset_prep, config['fps'], config['bold_delay'])
test_dataset_prep = preproc.bold_time_delay_align(test_dataset_prep, config['fps'], config['bold_delay'])


# torch dataset creation 
torch_dataset_train = torch_dataset.CreateDataset_eeg_fmri(train_dataset_prep, 
                                                            random_sample=config['random_subsample'], 
                                                            sample_per_epoch=config['sample_per_epoch'])

torch_dataset_test = torch_dataset.CreateDataset_eeg_fmri(test_dataset_prep, 
                                                            random_sample=False, 
                                                            sample_per_epoch=None)

# init dataloaders for training
train_loader = torch.utils.data.DataLoader(torch_dataset_train, **params_train)
val_loader = torch.utils.data.DataLoader(torch_dataset_test, **params_val)

Size of train dataset: eeg (30, 216900) | fmri (21, 216900)
Size of test dataset: eeg (30, 60000) | fmri (21, 60000)
Size of eeg features (30, 16, 216900) | fmri (21, 216900)
Size of eeg features (30, 16, 60000) | fmri (21, 60000)


In [11]:
model = autoencoder_v3.AutoEncoder1D(**hp_autoencoder)

loss_func = train_utils.make_mse_loss()
train_step = train_utils.train_step

optimizer = optim.Adam(model.parameters(), 
                       lr=config['lr'], 
                       weight_decay=config['weight_decay'])


print(summary(model, torch.zeros(4, config['n_channels'], len(config['freqs']), 1024), show_input=False))


-------------------------------------------------------------------------
      Layer (type)          Output Shape         Param #     Tr. Param #
          Conv2d-1     [4, 16, 16, 1024]             496             496
          Conv1d-2         [4, 32, 1024]           8,224           8,224
         Block1D-3          [4, 32, 128]           5,536           5,536
         Block1D-4           [4, 64, 16]          19,264          19,264
         Block1D-5            [4, 64, 4]          21,312          21,312
         Block1D-6            [4, 64, 4]          21,312          21,312
          Conv1d-7            [4, 21, 4]           1,365           1,365
        Upsample-8         [4, 21, 1024]               0               0
Total params: 77,509
Trainable params: 77,509
Non-trainable params: 0
-------------------------------------------------------------------------


/opt/conda/envs/myenv_torch/lib/python3.9/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


# Model training

In [12]:
n_runs = 3

for i in range(n_runs):
    
    model = autoencoder_v3.AutoEncoder1D(**hp_autoencoder)

    loss_func = train_utils.make_mse_loss()
    train_step = train_utils.train_step

    optimizer = optim.Adam(model.parameters(), 
                       lr=config['lr'], 
                       weight_decay=config['weight_decay'])
    
    
    parameters = {
        'EPOCHS': 50,
        'model': model, 
        'train_loader': train_loader, 
        'val_loader': val_loader, 
        'loss_function': loss_func,
        'train_step': train_step,
        'optimizer': optimizer, 
        'device': 'cuda', 
        'raw_test_data': test_dataset_prep,
        'show_info': 1, 
        'num_losses': 5,
        'labels': labels_roi,
        'inference_function': inference.model_inference_function, 
        'to_many': config['to_many']
    }



    path_to_save_wandb = 'common/koval_alvi/Checkpoints/wandb_brain'
    
    
    with wandb.init(project="eeg_fmri", config=config, save_code=True):
        
        if i == 0: 
            exp_name = wandb.run.name
        
        wandb.run.name = exp_name +'_run_' + str(i)
        
        print(config)
        print(parameters['model'])
        print(summary(model, torch.zeros(4, config['n_channels'], len(config['freqs']), config['WINDOW_SIZE']), show_input=False))
        
        model = train_utils.wanb_train_regression(**parameters)
        

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: koval_alvi (use `wandb login --relogin` to force relogin)


{'n_electrodes': 30, 'n_freqs': 16, 'n_channels_out': 21, 'n_res_block': 0, 'channels': [32, 32, 64, 64], 'kernel_sizes': [15, 11, 5, 5], 'strides': [8, 8, 4], 'dataset_name': 'CWL', 'fps': 1000, 'n_channels': 30, 'n_roi': 21, 'test_sec': 60, 'freqs': array([  2.        ,   8.53333333,  15.06666667,  21.6       ,
        28.13333333,  34.66666667,  41.2       ,  47.73333333,
        54.26666667,  60.8       ,  67.33333333,  73.86666667,
        80.4       ,  86.93333333,  93.46666667, 100.        ]), 'bold_delay': 5, 'to_many': True, 'random_subsample': True, 'sample_per_epoch': 25600, 'WINDOW_SIZE': 1024, 'optimizer': 'adam', 'lr': 0.001, 'weight_decay': 1e-05, 'batch_size': 512, 'loss_function': 'mse', 'model_type': '1D_CNN_AE_changed'}
AutoEncoder1D(
  (spatial_reduce_2d): Conv2d(30, 16, kernel_size=(1, 1), stride=(1, 1))
  (spatial_reduce): Conv1d(256, 32, kernel_size=(1,), stride=(1,))
  (downsample_blocks): ModuleList(
    (0): Block1D(
      (downsample): AvgPool1d(kernel_size=(

/opt/conda/envs/myenv_torch/lib/python3.9/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


.................................................
Epoch 2 train loss_0 : 0.98 val loss_0 : 1.22 train loss_1 : 0.029 val loss_1 : -0.0755 
.

/opt/conda/envs/myenv_torch/lib/python3.9/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


...

/opt/conda/envs/myenv_torch/lib/python3.9/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


.................................................
Epoch 41 train loss_0 : 0.947 val loss_0 : 1.24 train loss_1 : 0.26 val loss_1 : -0.00607 
.

/opt/conda/envs/myenv_torch/lib/python3.9/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


.........................

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# 